# Hyper-parameter Optimization Results

To effectively decide which parameters to use on our models, we have to evaluate the HPO results on the GoEmotions dataset.

In [1]:
import glob
import json
import pandas as pd

results = []
paths = glob.glob('../output/*/*/*.json', recursive=True)
for path in paths:
    with open(path) as fp:
        result_dict = json.load(fp)
        result_tuple = (path, result_dict)
        results.append(result_tuple)

results_df = pd.DataFrame(results, columns=['Path', 'Dict'])

In [2]:
EXPERIMENT_METRICS = ['macro_f1', 'micro_f1']
EXPERIMENT_COLUMNS = ['Dataset', 'ModelType', 'Experiment']
INTERNAL_METRICS = ['precision', 'recall']
WEIGHT_FIELD = 'support'


def parse_path(path):
    dataset, model_type, experiment = path.split('/')[-3:]
    experiment = experiment.split('.')[0]
    return dataset, model_type, experiment


def build_experiment_columns(dataframe):
    path_parts = [parse_path(path) for path in dataframe.Path]
    for i, col in enumerate(EXPERIMENT_COLUMNS):
        dataframe[col] = [parts[i] for parts in path_parts]
    return dataframe


def extract_macro_stats(dataframe, metrics=EXPERIMENT_METRICS):
    try:
        split_names = dataframe['Dict'][0]['config']['data_config']['split_names']
    except IndexError:
        split_names = ['train', 'valid', 'test']
    for split in split_names:
        for metric in metrics:
            new_col_name = '{}_{}'.format(split, metric)
            dataframe[new_col_name] = [exp_dict['results'][split][metric] for exp_dict in dataframe['Dict']]
    return dataframe


def extract_category_stats(dataframe, metrics=INTERNAL_METRICS, weight_field=WEIGHT_FIELD):
    try:
        split_names = dataframe['Dict'][0]['config']['data_config']['split_names']
    except IndexError:
        split_names = ['train', 'valid', 'test']
    for split in split_names:
        for metric in metrics:
            new_col_name = '{}_{}'.format(split, metric)
            metric_dict = []
            for exp_dict in dataframe['Dict']:
                label_values = list(exp_dict['results'][split]['labels'].values())
                global_weight = sum(label_results[weight_field] for label_results in label_values)
                metric_score = sum(label_results[weight_field] * label_results[metric] / global_weight
                                   for label_results in label_values)
                metric_dict.append(metric_score)
            dataframe[new_col_name] = metric_dict
    return dataframe


def extract_model_details(dataframe):
    dataframe['Extractor'] = [exp_dict['config']['extractor_config']['ex_type'] for exp_dict in dataframe['Dict']]
    dataframe['Model'] = [exp_dict['config']['model_config']['model_name'] for exp_dict in dataframe['Dict']]
    return dataframe


def parse_df(dataframe):
    dataframe = build_experiment_columns(dataframe)
    dataframe = extract_model_details(dataframe)
    dataframe = extract_macro_stats(dataframe)
    dataframe = extract_category_stats(dataframe)
    return dataframe


In [3]:
parsed_df = parse_df(results_df)
parsed_df['Dataset'] = [row.Dataset if row.Dataset != 'large-grid' else row.Path.split('/')[2]
                        for i, row in parsed_df[['Dataset', 'Path']].iterrows()]
parsed_df['SplitType'] = [d['config']['data_config']['split_mode'] for d in parsed_df.Dict]


parsed_df.sort_values('valid_macro_f1')

,Path,Dict,Dataset,ModelType,Experiment,Extractor,Model,train_macro_f1,train_micro_f1,valid_macro_f1,valid_micro_f1,test_macro_f1,test_micro_f1,train_precision,train_recall,valid_precision,valid_recall,test_precision,test_recall,SplitType
1158,../output/Vent/neural/52ce66a7a5c985ed9d0d9368...,{'config': {'data_config': {'raw_path': 'prepr...,Vent,neural,52ce66a7a5c985ed9d0d9368b83743dd,bert,dnnpool,0.017744,0.029671,0.015231,0.027161,0.015436,0.028582,0.017805,0.511462,0.021659,0.435901,0.016890,0.412615,column
478,../output/Vent/replica-fractions-with-test/89a...,{'config': {'data_config': {'raw_path': 'prepr...,Vent,replica-fractions-with-test,89a5fdad2426a06688da1d8aaf421701,fasttext,dnnpool,0.017636,0.030075,0.016840,0.031353,0.018176,0.032292,0.018164,0.667753,0.018455,0.617188,0.020165,0.682583,column
1104,../output/Vent/neural/3d433974ce46ca09abbf59ef...,{'config': {'data_config': {'raw_path': 'prepr...,Vent,neural,3d433974ce46ca09abbf59ef50b23c04,fasttext,dnnpool,0.020536,0.031562,0.017813,0.027809,0.016795,0.027072,0.020763,0.592551,0.022148,0.380454,0.021086,0.247270,column
1111,../output/Vent/neural/cd0b01d095b96f8a7b3f8935...,{'config': {'data_config': {'raw_path': 'prepr...,Vent,neural,cd0b01d095b96f8a7b3f8935c3b125ec,fasttext,dnnpool,0.020063,0.028314,0.017923,0.026142,0.016516,0.026958,0.020251,0.564646,0.019656,0.373224,0.021301,0.231763,column
409,../output/Vent/replica-fractions-with-test/7dd...,{'config': {'data_config': {'raw_path': 'prepr...,Vent,replica-fractions-with-test,7dd835eb82433f1a5d6aa47f80faae7f,fasttext,dnnpool,0.018884,0.031018,0.018704,0.031151,0.019195,0.032364,0.017210,0.706341,0.017329,0.649677,0.018589,0.686943,column
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4361,../output/GoEmotions/neural/812213d5b056aac532...,{'config': {'data_config': {'raw_path': 'prepr...,GoEmotions,neural,812213d5b056aac532335d8800486c5d,bert,dnnpool,0.593322,0.634571,0.515087,0.559888,0.499480,0.556102,0.609850,0.668141,0.510231,0.642633,0.519813,0.621899,column
4147,../output/GoEmotions/neural/7e372907c3f585085f...,{'config': {'data_config': {'raw_path': 'prepr...,GoEmotions,neural,7e372907c3f585085f6d037c3849ca63,bert,dnnpool,0.556554,0.597283,0.515675,0.558934,0.510514,0.555689,0.553376,0.662760,0.514209,0.646395,0.515199,0.634065,column
5154,../output/GoEmotions/neural/dd9f3603b2210ed1f5...,{'config': {'data_config': {'raw_path': 'prepr...,GoEmotions,neural,dd9f3603b2210ed1f58e963a99102c59,bert,dnnpool,0.556554,0.597283,0.515675,0.558934,0.510514,0.555689,0.553376,0.662760,0.514209,0.646395,0.515199,0.634065,column
2930,../output/GoEmotions/neural/f7cacae6393f216913...,{'config': {'data_config': {'raw_path': 'prepr...,GoEmotions,neural,f7cacae6393f2169131c6f7c394e5403,bert,dnnpool,0.564260,0.612822,0.516185,0.562844,0.498440,0.559146,0.580436,0.661801,0.513887,0.652508,0.522387,0.622689,column


In [4]:
df_column_experiments = parsed_df[parsed_df.SplitType == 'column']
keyed_groups = df_column_experiments.groupby(['Dataset', 'ModelType', 'Extractor', 'Model'])
best_valid_df = keyed_groups.valid_macro_f1.max().to_frame().reset_index()

results_df = parsed_df.merge(best_valid_df, on=['Dataset', 'ModelType', 'Extractor', 'Model', 'valid_macro_f1'], how='inner')

final_df = results_df[['Dict', 'Dataset', 'ModelType', 'Extractor', 'Model', 'test_macro_f1', 'test_micro_f1', 'test_precision', 'test_recall']].round(2)
final_df = final_df.drop_duplicates(['Dataset', 'Extractor', 'Model']).sort_values(['Dataset', 'Extractor', 'Model'])

def correct_family(family):
    if family.startswith('replica'):
        return 'neural'
    if family == 'dummy':
        return 'classic'
    return family
    
final_df.ModelType = [correct_family(fam) for fam in final_df.ModelType.tolist()]
final_df

,Dict,Dataset,ModelType,Extractor,Model,test_macro_f1,test_micro_f1,test_precision,test_recall
110,{'config': {'data_config': {'raw_path': 'prepr...,GoEmotions,neural,bert,dnnpool,0.49,0.55,0.52,0.60
104,{'config': {'data_config': {'raw_path': 'prepr...,GoEmotions,neural,bert,lstm,0.48,0.56,0.52,0.63
91,{'config': {'data_config': {'raw_path': 'prepr...,GoEmotions,classic,bow,naivebayes,0.35,0.46,0.43,0.53
87,{'config': {'data_config': {'raw_path': 'prepr...,GoEmotions,classic,bow,rf,0.47,0.53,0.50,0.59
93,{'config': {'data_config': {'raw_path': 'prepr...,GoEmotions,classic,bow,sgd,0.46,0.53,0.49,0.61
105,{'config': {'data_config': {'raw_path': 'prepr...,GoEmotions,neural,fasttext,dnnpool,0.42,0.49,0.45,0.61
111,{'config': {'data_config': {'raw_path': 'prepr...,GoEmotions,neural,fasttext,lstm,0.45,0.54,0.51,0.59
89,{'config': {'data_config': {'raw_path': 'prepr...,GoEmotions,classic,tfidf,naivebayes,0.33,0.44,0.43,0.49
86,{'config': {'data_config': {'raw_path': 'prepr...,GoEmotions,classic,tfidf,rf,0.46,0.52,0.48,0.60
88,{'config': {'data_config': {'raw_path': 'prepr...,GoEmotions,classic,tfidf,sgd,0.47,0.53,0.49,0.60


# Generate the final experiment configuration

Create the jsons with a number of seeds for the final experiment with multiple runs.

In [6]:
import sys
import copy
import json
sys.path.append('../src')
from config import ExperimentConfig

NUM_EXPERIMENTS = 5

for experiment in range(NUM_EXPERIMENTS):
    for cfg in final_df[final_df.Dataset == 'GoEmotions'].Dict:
        # Clone the config
        # Not needed for seeds, but if we change something in the future we'll be grateful
        cfg = copy.deepcopy(cfg)
        cfg = cfg['config']
        cfg['seed'] = experiment
        model_family = correct_family(cfg['output_path'].split('/')[-2])
        cfg['output_path'] = f'output/GoEmotions/replica/'
        
        # Build the experiment object
        exp_cfg = ExperimentConfig.from_dict(cfg)        
        as_json = json.dumps(exp_cfg._as_flat_dict(), indent=2)
        exp_hash = exp_cfg.hash()
        
        # Save as json
        save_path = f'../configs/GoEmotions/replica/{model_family}/{exp_hash}.json' 
        with open(save_path, 'w') as f:
            f.write(as_json)

# Generate the experiment configuration for Sampled Robust Vent

In [7]:
import sys
import copy
import json
sys.path.append('../src')
from config import ExperimentConfig

NUM_EXPERIMENTS = 5

for experiment in range(NUM_EXPERIMENTS):
    for cfg in final_df[final_df.Dataset == 'Vent'].Dict:
        # Clone the config
        # Not needed for seeds, but if we change something in the future we'll be grateful
        cfg = copy.deepcopy(cfg)
        cfg = cfg['config']
        cfg['seed'] = experiment
        model_family = correct_family(cfg['output_path'].split('/')[-2])
        cfg['output_path'] = f'output/Vent/replica/'
        
        # Build the experiment object
        exp_cfg = ExperimentConfig.from_dict(cfg)
        as_json = json.dumps(exp_cfg._as_flat_dict(), indent=2)
        exp_hash = exp_cfg.hash()
        
        # Save as json
        save_path = f'../configs/Vent/replica/{model_family}/{exp_hash}.json' 
        with open(save_path, 'w') as f:
            f.write(as_json)

# Sampled Vent with random (rather than temporal) splits

In [8]:
import sys
import copy
import json
sys.path.append('../src')
from config import ExperimentConfig

NUM_EXPERIMENTS = 5

for experiment in range(NUM_EXPERIMENTS):
    for cfg in final_df[final_df.Dataset == 'Vent'].Dict:
        # Clone the config
        # Not needed for seeds, but if we change something in the future we'll be grateful
        cfg = copy.deepcopy(cfg)
        cfg = cfg['config']
        cfg['seed'] = experiment
        model_family = correct_family(cfg['output_path'].split('/')[-2])
        cfg['output_path'] = f'output/Vent/replica-random/'
        cfg['data_config']['split_mode'] = 'random'
        cfg['data_config']['cache_path'] = 'preprocessed/vent-split-random-cache/'
        
        # Build the experiment object
        exp_cfg = ExperimentConfig.from_dict(cfg)        
        as_json = json.dumps(exp_cfg._as_flat_dict(), indent=2)
        exp_hash = exp_cfg.hash()
        
        # Save as json
        save_path = f'../configs/Vent/replica-random/{model_family}/{exp_hash}.json' 
        with open(save_path, 'w') as f:
            f.write(as_json)

# Complete robust Vent with random and temporal splits

We reduce the number of epochs from 5, 10, 20, 40, 60 to `1 + round(epochs / 30)`, e.g. 1, 1, 2, 2, 3. Since we are only mapping the winning configuration, the duped values will not be a problem.

In [9]:
import sys
import copy
import json
sys.path.append('../src')
from config import ExperimentConfig

NUM_EXPERIMENTS = 5

for experiment in range(NUM_EXPERIMENTS):
    for cfg in final_df[final_df.Dataset == 'Vent'].Dict:
        # Clone the config
        # Not needed for seeds, but if we change something in the future we'll be grateful
        cfg = copy.deepcopy(cfg)
        cfg = cfg['config']
        cfg['seed'] = experiment
        model_family = correct_family(cfg['output_path'].split('/')[-2])
        cfg['output_path'] = f'output/Vent/replica-full/'
        cfg['data_config']['raw_path'] = 'preprocessed/vent-robust-splits.parquet'
        cfg['data_config']['cache_path'] = 'preprocessed/vent-split-robust-cache/'
        if 'num_epochs' in cfg['model_config']['model_conf']:
            num_epochs = cfg['model_config']['model_conf']['num_epochs']
            if num_epochs > 4:
                cfg['model_config']['model_conf']['num_epochs'] = 1 + round(num_epochs / 30)

        if 'n_estimators_per_chunk' in cfg['model_config']['model_conf']:
            num_estimators = cfg['model_config']['model_conf']['n_estimators_per_chunk']
            if num_estimators > 500:
                cfg['model_config']['model_conf']['n_estimators_per_chunk'] = num_estimators // 30
                    
        # Build the experiment object
        exp_cfg = ExperimentConfig.from_dict(cfg)        
        as_json = json.dumps(exp_cfg._as_flat_dict(), indent=2)
        exp_hash = exp_cfg.hash()
        
        # Save as json
        save_path = f'../configs/Vent/replica-full/{model_family}/{exp_hash}.json' 
        with open(save_path, 'w') as f:
            f.write(as_json)

In [10]:
import sys
import copy
import json
sys.path.append('../src')
from config import ExperimentConfig

NUM_EXPERIMENTS = 5

for experiment in range(NUM_EXPERIMENTS):
    for cfg in final_df[final_df.Dataset == 'Vent'].Dict:
        # Clone the config
        # Not needed for seeds, but if we change something in the future we'll be grateful
        cfg = copy.deepcopy(cfg)
        cfg = cfg['config']
        cfg['seed'] = experiment
        model_family = correct_family(cfg['output_path'].split('/')[-2])
        cfg['output_path'] = f'output/Vent/replica-full-random/'
        cfg['data_config']['split_mode'] = 'random'
        cfg['data_config']['raw_path'] = 'preprocessed/vent-robust-splits.parquet'
        cfg['data_config']['cache_path'] = 'preprocessed/vent-split-robust-random-cache/'
        if 'num_epochs' in cfg['model_config']['model_conf']:
            num_epochs = cfg['model_config']['model_conf']['num_epochs']
            if num_epochs > 4:
                cfg['model_config']['model_conf']['num_epochs'] = 1 + round(num_epochs / 30)

        if 'n_estimators_per_chunk' in cfg['model_config']['model_conf']:
            num_estimators = cfg['model_config']['model_conf']['n_estimators_per_chunk']
            if num_estimators > 500:
                cfg['model_config']['model_conf']['n_estimators_per_chunk'] = num_estimators // 30
                    
        # Build the experiment object
        exp_cfg = ExperimentConfig.from_dict(cfg)        
        as_json = json.dumps(exp_cfg._as_flat_dict(), indent=2)
        exp_hash = exp_cfg.hash()
        
        # Save as json
        save_path = f'../configs/Vent/replica-full-random/{model_family}/{exp_hash}.json' 
        with open(save_path, 'w') as f:
            f.write(as_json)

# Final experiments — sampling fractions and temporal inversion

In [12]:
import sys
import copy
import json
sys.path.append('../src')
from config import ExperimentConfig

NUM_EXPERIMENTS = 5

for experiment in range(NUM_EXPERIMENTS):
    for cfg in final_df[final_df.Dataset == 'Vent'].Dict:
        # Clone the config
        # Not needed for seeds, but if we change something in the future we'll be grateful
        cfg = copy.deepcopy(cfg)
        cfg = cfg['config']
        cfg['seed'] = experiment
        model_family = correct_family(cfg['output_path'].split('/')[-2])
        cfg['output_path'] = f'output/Vent/replica-fractions'
        cfg['data_config']['raw_path'] = 'preprocessed/vent-robust-splits-backwards.parquet'
        cfg['data_config']['cache_path'] = 'preprocessed/vent-split-robust-backwards-cache/'
        if 'num_epochs' in cfg['model_config']['model_conf']:
            num_epochs = cfg['model_config']['model_conf']['num_epochs']
            if num_epochs > 4:
                cfg['model_config']['model_conf']['num_epochs'] = 1 + round(num_epochs / 30)

        if 'n_estimators_per_chunk' in cfg['model_config']['model_conf']:
            num_estimators = cfg['model_config']['model_conf']['n_estimators_per_chunk']
            if num_estimators > 500:
                cfg['model_config']['model_conf']['n_estimators_per_chunk'] = num_estimators // 30
                    
        # Build the experiment object
        exp_cfg = ExperimentConfig.from_dict(cfg)        
        as_json = json.dumps(exp_cfg._as_flat_dict(), indent=2)
        exp_hash = exp_cfg.hash()
        
        # Save as json
        save_path = f'../configs/Vent/replica-fractions/{model_family}/{exp_hash}.json' 
        with open(save_path, 'w') as f:
            f.write(as_json)

In [15]:
import sys
import copy
import json
sys.path.append('../src')
from config import ExperimentConfig

NUM_EXPERIMENTS = 5

for experiment in range(NUM_EXPERIMENTS):
    for pct in [5, 10, 20, 40]:
        for cfg in final_df[final_df.Dataset == 'Vent'].Dict:
            # Clone the config
            # Not needed for seeds, but if we change something in the future we'll be grateful
            cfg = copy.deepcopy(cfg)
            cfg = cfg['config']
            cfg['seed'] = experiment
            model_family = correct_family(cfg['output_path'].split('/')[-2])
            cfg['output_path'] = f'output/Vent/replica-fractions/'
            cfg['data_config']['raw_path'] = f'preprocessed/vent-robust-splits-{pct}-pct.parquet'
            cfg['data_config']['cache_path'] = f'preprocessed/vent-split-robust-cache-{pct}-pct/'
            if 'num_epochs' in cfg['model_config']['model_conf']:
                num_epochs = cfg['model_config']['model_conf']['num_epochs']
                if num_epochs > 4:
                    cfg['model_config']['model_conf']['num_epochs'] = 1 + round(num_epochs / 30)

            if 'n_estimators_per_chunk' in cfg['model_config']['model_conf']:
                num_estimators = cfg['model_config']['model_conf']['n_estimators_per_chunk']
                if num_estimators > 500:
                    cfg['model_config']['model_conf']['n_estimators_per_chunk'] = num_estimators // 30

            # Build the experiment object
            exp_cfg = ExperimentConfig.from_dict(cfg)        
            as_json = json.dumps(exp_cfg._as_flat_dict(), indent=2)
            exp_hash = exp_cfg.hash()

            # Save as json
            save_path = f'../configs/Vent/replica-fractions/{model_family}/{exp_hash}.json' 
            with open(save_path, 'w') as f:
                f.write(as_json)

# Final experiments — Fractions with full test split

In [11]:
final_df

,Dict,Dataset,ModelType,Extractor,Model,test_macro_f1,test_micro_f1,test_precision,test_recall
91,{'config': {'data_config': {'raw_path': 'prepr...,GoEmotions,neural,bert,dnnpool,0.49,0.55,0.52,0.60
85,{'config': {'data_config': {'raw_path': 'prepr...,GoEmotions,neural,bert,lstm,0.48,0.56,0.52,0.63
72,{'config': {'data_config': {'raw_path': 'prepr...,GoEmotions,classic,bow,naivebayes,0.35,0.46,0.43,0.53
68,{'config': {'data_config': {'raw_path': 'prepr...,GoEmotions,classic,bow,rf,0.47,0.53,0.50,0.59
74,{'config': {'data_config': {'raw_path': 'prepr...,GoEmotions,classic,bow,sgd,0.46,0.53,0.49,0.61
86,{'config': {'data_config': {'raw_path': 'prepr...,GoEmotions,neural,fasttext,dnnpool,0.42,0.49,0.45,0.61
92,{'config': {'data_config': {'raw_path': 'prepr...,GoEmotions,neural,fasttext,lstm,0.45,0.54,0.51,0.59
70,{'config': {'data_config': {'raw_path': 'prepr...,GoEmotions,classic,tfidf,naivebayes,0.33,0.44,0.43,0.49
67,{'config': {'data_config': {'raw_path': 'prepr...,GoEmotions,classic,tfidf,rf,0.46,0.52,0.48,0.60
69,{'config': {'data_config': {'raw_path': 'prepr...,GoEmotions,classic,tfidf,sgd,0.47,0.53,0.49,0.60


In [5]:
import sys
import copy
import json
sys.path.append('../src')
from config import ExperimentConfig

NUM_EXPERIMENTS = 5

for experiment in range(NUM_EXPERIMENTS):
    for pct in [5, 10, 20, 40, 60, 80, 100]:
        for i, row in final_df[final_df.Dataset == 'Vent'].iterrows():
            # Clone the config
            # Not needed for seeds, but if we change something in the future we'll be grateful
            cfg = row.Dict
            cfg = copy.deepcopy(cfg)
            cfg = cfg['config']
            cfg['seed'] = experiment
            model_family = row.ModelType
            cfg['output_path'] = f'output/Vent/replica-fractions-with-test/'
            cfg['data_config']['raw_path'] = f'preprocessed/vent-robust-splits-{pct}-pct.parquet'
            cfg['data_config']['cache_path'] = f'preprocessed/vent-split-robust-cache-{pct}-pct-with-test/'
            if 'num_epochs' in cfg['model_config']['model_conf']:
                num_epochs = cfg['model_config']['model_conf']['num_epochs']
                if num_epochs > 4:
                    cfg['model_config']['model_conf']['num_epochs'] = 1 + round(num_epochs / 30)

            if 'n_estimators_per_chunk' in cfg['model_config']['model_conf']:
                num_estimators = cfg['model_config']['model_conf']['n_estimators_per_chunk']
                if num_estimators > 500:
                    cfg['model_config']['model_conf']['n_estimators_per_chunk'] = num_estimators // 30

            # Store the model for the first experiment
            if experiment == 0:
                cfg['model_path'] = 'models/Vent/fractions-with-test/'
                    
            # Build the experiment object
            exp_cfg = ExperimentConfig.from_dict(cfg)        
            as_json = json.dumps(exp_cfg._as_flat_dict(), indent=2)
            exp_hash = exp_cfg.hash()

            # Save as json
            save_path = f'../configs/Vent/replica-fractions-with-test/{model_family}/{exp_hash}.json' 
            with open(save_path, 'w') as f:
                f.write(as_json)